In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_2017028/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.7)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_9_28,0.990114,0.769933,0.744306,0.975421,0.872633,0.041488,1.538460,0.186979,0.021948,0.104464,0.711752,0.203685,1.004237,0.212356,166.364719,263.874785,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
1,model_1_9_29,0.990114,0.769954,0.743864,0.974652,0.872017,0.041489,1.538321,0.187302,0.022635,0.104969,0.710807,0.203688,1.004237,0.212360,166.364659,263.874725,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
2,model_1_9_27,0.990114,0.769911,0.744741,0.976185,0.873242,0.041491,1.538606,0.186661,0.021266,0.103964,0.712709,0.203694,1.004237,0.212366,166.364542,263.874608,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
3,model_1_9_30,0.990113,0.769974,0.743414,0.973878,0.871395,0.041495,1.538187,0.187632,0.023326,0.105479,0.709877,0.203703,1.004237,0.212375,166.364362,263.874428,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
4,model_1_9_26,0.990111,0.769888,0.745168,0.976943,0.873845,0.041500,1.538757,0.186349,0.020589,0.103469,0.713682,0.203715,1.004238,0.212388,166.364126,263.874192,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,model_1_2_4,0.892591,0.682260,-2.174573,0.962328,0.810274,0.450772,2.124728,0.582859,0.043514,0.313186,1.454153,0.671395,1.046032,0.699978,161.593588,259.103654,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
396,model_1_2_3,0.890334,0.680619,-2.257362,0.962758,0.805821,0.460244,2.135700,0.598059,0.043016,0.320538,1.462785,0.678413,1.047000,0.707294,161.551996,259.062062,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
397,model_1_2_2,0.888013,0.678940,-2.342287,0.962860,0.801134,0.469985,2.146928,0.613652,0.042899,0.328276,1.471765,0.685554,1.047994,0.714740,161.510111,259.020177,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
398,model_1_2_1,0.885627,0.677222,-2.429407,0.962630,0.796208,0.479999,2.158417,0.629647,0.043164,0.336406,1.481098,0.692819,1.049017,0.722314,161.467944,258.978010,"Hidden Size=[15, 2], regularizer=0.02, learnin..."
